In [ ]:
!pip install --pre azure-cosmos

In [ ]:
import numpy as np
import random
import time
import uuid

from azure.cosmos import CosmosClient, PartitionKey, exceptions

In [ ]:
# Initialize the Cosmos client
COSMOS_ENDPOINT = "<<TODO: endpoint>>"
COSMOS_KEY = '<<TODO: primary_key>>'

cosmos_client = CosmosClient(COSMOS_ENDPOINT, COSMOS_KEY)

In [ ]:
# Create reference variables to existing database and containers
db_id = 'MoviesDB'
c_cartitem_id = 'CartItems'
c_categories_id = 'Categories'
c_events_id = 'Events'
c_items_id = 'Items'
c_orders_id = 'Orders'
c_users_id = 'Users'

db_client = cosmos_client.get_database_client(db_id)
c_cartitem_client = db_client.get_container_client(c_cartitem_id)
c_categories_client = db_client.get_container_client(c_categories_id)
c_events_client = db_client.get_container_client(c_events_id)
c_items_client = db_client.get_container_client(c_items_id)
c_orders_client = db_client.get_container_client(c_orders_id)
c_users_client = db_client.get_container_client(c_users_id)

These are the list of requests that we have along with their estimated number of requests per second.

```
	• Retrieve list of top 10 movies by popularity (home page): ~500 requests/second
	• Retrieve list of categories: ~500 requests/second
	• Filter movies by category: ~200 requests/second
	• Retrieve details for a specific movie: ~160 requests/second
	• Retrieve list of last 10 movies by release date: ~75 requests/second
	• Page through lists of movies: ~30 requests/second
	• Retrieve orders with details showing products with quantities: ~10 requests/second
	• Add a movie to the shopping cart: ~5 requests/second
	• Complete a purchase transaction: ~2 requests/second
```

In [ ]:
# Get RUs for query "Retrieve list of top 10 movies by popularity (home page)"

query = "SELECT TOP 10 * FROM c ORDER BY c.Popularity"

items = list(c_items_client.query_items(
    query=query,
    enable_cross_partition_query=True
))

top_10_movies_ru = c_items_client.client_connection.last_response_headers['x-ms-request-charge']

print(f'Query returned {len(items)} items. Operation consumed {top_10_movies_ru} request units')

In [ ]:
# Get RUs for query "Retrieve list of categories"

query = "SELECT * FROM c"

categories = list(c_categories_client.query_items(
    query=query,
    enable_cross_partition_query=True
))

categories_list_ru = c_categories_client.client_connection.last_response_headers['x-ms-request-charge']

print(f'Query returned {len(categories)} items. Operation consumed {categories_list_ru} request units')

In [ ]:
# Get RUs for query "Filter movies by category"

rus = []
for cat in categories:
    cat_id = cat['CategoryId']
    query = f"SELECT * FROM c WHERE c.CategoryId = {cat_id}"
    items = list(c_items_client.query_items(
        query=query,
        partition_key=cat_id
    ))
    ru = c_items_client.client_connection.last_response_headers['x-ms-request-charge']
    rus.append(float(ru))

# Taking average of RUs
filter_movie_ru = np.mean(rus)
print(f'Operation consumed an average of {filter_movie_ru} request units')

In [ ]:
# Get RUs for query "Retrieve details for a specific movie"

rus = []
for item in items[0:20]:
    item_id = item['ItemId']
    query = f"SELECT * FROM c WHERE c.ItemId = {item_id}"
    items = list(c_items_client.query_items(
        query=query,
        enable_cross_partition_query=True
    ))
    ru = c_items_client.client_connection.last_response_headers['x-ms-request-charge']
    rus.append(float(ru))

# Taking average of RUs
movie_details_ru = np.mean(rus)
print(f'Operation consumed an average of {movie_details_ru} request units')

In [ ]:
# Get RUs for query "Retrieve list of last 10 movies by release date"

query = "SELECT TOP 10 * FROM c ORDER BY c.ReleaseDate DESC"

items = list(c_items_client.query_items(
    query=query,
    enable_cross_partition_query=True
))

latest_10_movies_ru = c_items_client.client_connection.last_response_headers['x-ms-request-charge']

print(f'Query returned {len(items)} items. Operation consumed {latest_10_movies_ru} request units')

In [ ]:
# Get RUs for query "Page through lists of movies"

rus = []

query = "SELECT * FROM c"

items_iterable = c_items_client.query_items(
    query=query,
    enable_cross_partition_query=True,
    feed_options = {
        'maxItemCount': 10
    }
)

for _ in range(0, 10):
    items_iterable.next()
    ru = c_items_client.client_connection.last_response_headers['x-ms-request-charge']
    rus.append(float(ru))

paging_movies_ru = np.mean(rus)

print(f'Operation consumed an average of {paging_movies_ru} request units')

In [ ]:
# Get RUs for query "Retrieve orders with details showing products with quantities"

query = "SELECT * FROM c"

orders = list(c_orders_client.query_items(
    query=query,
    enable_cross_partition_query=True
))

orders_list_ru = c_orders_client.client_connection.last_response_headers['x-ms-request-charge']

print(f'Query returned {len(orders)} items. Operation consumed {orders_list_ru} request units')

In [ ]:
# Get RUs for query "Add a movie to the shopping cart"

item_id = items[0]['ItemId']

movie_item = {
    'id': str(uuid.uuid4()),
    'CartItemId': str(uuid.uuid4()),
    'CartId': 1,
    'ItemId': item_id,
    'Quantity': 9,
    'DateCreated': int(round(time.time() * 1000))
}
c_cartitem_client.create_item(body=movie_item)

add_cart_ru = c_cartitem_client.client_connection.last_response_headers['x-ms-request-charge']

print(f'Operation consumed {add_cart_ru} request units')

In [ ]:
# Get RUs for query "Complete a purchase transaction"

item_id = items[0]['ItemId']

order_item = {
    "OrderID": random.randint(200, 1000),
    "OrderDate": "2019-06-11T22:12:50.600",
    "FirstName": "Gregory",
    "LastName": "Middleton",
    "Address": "369 Milton Parkway",
    "City": "Madison",
    "State": "Texas",
    "PostalCode": "41549",
    "Country": "United States",
    "Phone": "3284807419",
    "Total": 41.99,
    "OrderDetails": [
        {
            "OrderDetailId": 636,
            "ProductId": 397527,
            "UnitPrice": 8.66,
            "Quantity": 1
        },
        {
            "OrderDetailId": 637,
            "ProductId": 397528,
            "UnitPrice": 8.07,
            "Quantity": 1
        },
        {
            "OrderDetailId": 638,
            "ProductId": 397529,
            "UnitPrice": 5.81,
            "Quantity": 1
        },
        {
            "OrderDetailId": 639,
            "ProductId": 397530,
            "UnitPrice": 8.07,
            "Quantity": 1
        },
        {
            "OrderDetailId": 640,
            "ProductId": 397531,
            "UnitPrice": 11.38,
            "Quantity": 1
        }
    ],
    "id": str(uuid.uuid4())
}
c_orders_client.create_item(body=order_item)

add_order_ru = c_orders_client.client_connection.last_response_headers['x-ms-request-charge']

print(f'Operation consumed {add_order_ru} request units')

In [ ]:
# Estimate RUs for queries specified in requirements
total_ru = 0
total_ru = total_ru + (float(top_10_movies_ru) * 500)
total_ru = total_ru + (float(categories_list_ru) * 500)
total_ru = total_ru + (float(filter_movie_ru) * 200)
total_ru = total_ru + (float(movie_details_ru) * 160)
total_ru = total_ru + (float(latest_10_movies_ru) * 75)
total_ru = total_ru + (float(paging_movies_ru) * 30)
total_ru = total_ru + (float(orders_list_ru) * 10)
total_ru = total_ru + (float(add_cart_ru) * 5)
total_ru = total_ru + (float(add_order_ru) * 2)

print(f'Total estimated RUs for the database is {total_ru}')